***3주차***

10개 이미지 부르는 코드

In [1]:
import numpy as np
import skimage
from skimage import io
from matplotlib import pyplot as plt
from skimage import data
from skimage.transform import resize
from skimage.metrics import peak_signal_noise_ratio, structural_similarity

In [2]:
#Gaussian Filter

def gaussian_kernel(k_size, sigma):
    size = k_size//2
    y, x = np.ogrid[-size:size+1, -size:size+1]
    filter = 1/(2*np.pi * (sigma**2)) * np.exp(-1 *(x**2 + y**2)/(2*(sigma**2)))
    sum = filter.sum()
    filter /= sum
    return filter

def padding(img, k_size):
    pad_size = k_size//2
    h, w, ch = img.shape
    
    res = np.zeros((h + (2*pad_size), w+(2*pad_size), ch), dtype=np.float)
    
    if pad_size == 0:
        res = img.copy()
    else:
        res[pad_size:-pad_size, pad_size:-pad_size] = img.copy()
    return res

def gaussian_filtering(img, k_size=5,sigma=4):
    h, w, ch = img.shape
    filter = gaussian_kernel(k_size, sigma)
    pad_img = padding(img,k_size)
    filtered_img = np.zeros((h, w, ch), dtype=np.float32)
    
    for ch in range(0, ch):
        for i in range(h):
            for j in range(w):
                filtered_img[i, j, ch] = np.sum(filter * pad_img[i:i+k_size, j:j+k_size, ch])

    return filtered_img

In [5]:
def median_filter(img, filter_size=(3, 3), stride=1):
    
    img_shape = np.shape(img) # 이미지 크기 가져오기

    # 결과 이미지의 형태를 계산합니다.
    # 여기서 filter_size와 stride를 고려하여 결과 이미지의 각 차원 크기를 계산합니다.
    result_shape = tuple(np.int64((np.array(img_shape[:2]) - np.array(filter_size)) / stride) + 1) + (img_shape[2],)

    # 결과 이미지를 저장할 배열을 초기화합니다.
    result = np.zeros(result_shape)

    # 이미지를 순회하면서 각 픽셀에 대해 메디안 필터를 적용합니다.
    for h in range(0, result_shape[0], stride):
        for w in range(0, result_shape[1], stride):
            for c in range(img_shape[2]):
                # 현재 위치에서 필터 크기만큼의 영역을 추출합니다.
                tmp = img[h:h + filter_size[0], w:w + filter_size[1], c].ravel()
                
                # 추출된 영역의 값을 정렬합니다.
                tmp = np.sort(tmp)

                # 정렬된 값 중 중앙값을 결과 이미지의 해당 위치에 할당합니다.
                result[h, w, c] = tmp[int(len(tmp) / 2)]

    return result


IndentationError: expected an indented block (413812175.py, line 3)

In [4]:
def median_gaussian(noisy_img, clean_img):
    median_img = median_filter(noisy_img)
    median_img_resized = resize(median_img, clean_img.shape, mode ='constant', anti_aliasing = True)
    median_img_resized = np.clip(median_img_resized, 0., 1.0)
    median_gaussian_img = gaussian_filtering(median_img_resized)
    return median_gaussian_img
    

*** 5개 noisy img denoising ***

In [21]:
name = ['baby', 'bagles', 'beach', 'book', 'dog']

for fn in name :
    _noisy = '_noisy'
    png = '.png'
    
    filename = fn + png
    clean_img = io.imread (filename)
    clean_img = clean_img[:,:,0:3]
    clean_img = clean_img/255.0
    
    filename = ""
    filename = fn + _noisy + png
    noisy_img = io.imread (filename)
    noisy_img = noisy_img[:,:,0:3]
    noisy_img = noisy_img/255.0
    
    median_gaussian_img = median_gaussian(noisy_img, clean_img)
    title = filename + " denoising\nPSNR : " + str(peak_signal_noise_ratio(clean_img, median_gaussian_img))
    print(title)

/Users/parkhyungwook/opt/anaconda3/envs/kucel/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from ipykernel import kernelapp as app
/Users/parkhyungwook/opt/anaconda3/envs/kucel/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: Inputs have mismatched dtype.  Setting data_range based on image_true.


baby_noisy.png denoising
PSNR : 24.782051847175314
bagles_noisy.png denoising
PSNR : 22.44781265451123
beach_noisy.png denoising
PSNR : 23.676198315018027
book_noisy.png denoising
PSNR : 23.972523194422784
dog_noisy.png denoising
PSNR : 23.913322494339514


*** 5개의 최적 ***

In [ ]:
# 전에 구했던 최적의 파라미터 값들이 최적일까라는 생각이든다 -> 질문을 해보자 
name = ['baby', 'bagles', 'beach', 'book', 'dog', 'girl_ani', 'lego', 'kitty', 'house', 'street']

for fn in name :
    _noisy = '_noisy'
    png = '.png'
    
    filename = fn + png
    clean_img = io.imread (filename)
    clean_img = clean_img[:,:,0:3]
    clean_img = clean_img/255.0
    
    filename = ""
    filename = fn + _noisy + png
    noisy_img = io.imread(filename)
    noisy_img = noisy_img[:,:,0:3]
    noisy_img = noisy_img/255.0
    
    title = filename + " denoising\n"
    print(title)
    for k in range(0,3):
        mfs = 3 + (k * 2)
        filter_size = (mfs,mfs)
        stride = 1
        median_img = median_filter(noisy_img, filter_size, stride)
        median_img_resized = resize(median_img, clean_img.shape, mode ='constant', anti_aliasing = True)
        median_img_resized = np.clip(median_img_resized, 0., 1.0)
        for i in range(0, 3):
            for j in range(0,3):
                k_size = 3 + 2 * i
                sigma = 3 * j + 1
                res = gaussian_filtering(median_img_resized, k_size = k_size, sigma = sigma)
                title = "stride 1, filter size : " + str(mfs) + "\nk size : " + str(k_size) + ",  sigma = " + str(sigma) + "\nPSNR : " + str(peak_signal_noise_ratio(clean_img, res))
                print(title)

*** 양방향 필터 ***

In [ ]:
def bilateral_filter(noisy_img, k_size=5, sigma_space=4, sigma_intensity=0.2):
    h, w, ch = noisy_img.shape
    bilateral_noisy_img = np.zeros((h, w, ch))
    spatial_filter = gaussian_kernel(k_size, sigma_space)
    for c in range(ch):
        for i in range(h):
            for j in range(w):
                intensity_center = noisy_img[i, j, c]
                weighted_sum = 0.0
                normalization_factor = 0.0
                for m in range(-k_size//2, k_size//2 + 1):
                    for n in range(-k_size//2, k_size//2 + 1):
                        i_neighbor = i + m
                        j_neighbor = j + n
                        
                        if 0 <= i_neighbor < h and 0 <= j_neighbor < w:
                            intensity_neighbor = noisy_img[i_neighbor, j_neighbor, c]
                            weight_intensity = np.exp(-(intensity_center - intensity_neighbor)**2 / (2 * sigma_intensity**2))
                            weight_spatial = spatial_filter[m + k_size//2, n + k_size//2]
                            weight = weight_intensity * weight_spatial
                            weighted_sum += intensity_neighbor * weight
                            normalization_factor += weight

                bilateral_noisy_img[i, j, c] = weighted_sum / normalization_factor
    
    return bilateral_noisy_img

*** Non Local Mean Filter ***

In [ ]:
def calculate_weight(i, j, k, l, W, image_padded, h_squared):
    """
    Calculate the weight for the pixel at position (i, j) compared to a pixel at position (k, l).
    """
    patch_i_j = image_padded[i:i+2*W+1, j:j+2*W+1]
    patch_k_l = image_padded[k:k+2*W+1, l:l+2*W+1]
    distance_squared = np.sum((patch_i_j - patch_k_l) ** 2)
    weight = np.exp(-distance_squared / h_squared)
    return weight

def non_local_means_filtering_channel(image_channel, W, sigma):
    """
    Apply Non-Local Means filtering to a single color channel of an image.
    
    Parameters:
        image_channel: 2D numpy array representing one color channel of the image
        W: Integer, search window size
        sigma: Float, standard deviation of the noise
    
    Returns:
        image_channel_filtered: 2D numpy array representing the filtered color channel
    """
    M, N = image_channel.shape
    h = sigma * np.sqrt(2 * W + 1)
    h_squared = h ** 2
    image_padded = np.pad(image_channel, ((W, W), (W, W)), 'reflect')
    image_channel_filtered = np.zeros((M, N))
    
    for i in range(M):
        for j in range(N):
            weights = np.zeros((2*W+1, 2*W+1))
            for k in range(i, i + 2*W + 1):
                for l in range(j, j + 2*W + 1):
                    if k < M + W and l < N + W:
                        weights[k-i, l-j] = calculate_weight(i+W, j+W, k, l, W, image_padded, h_squared)
            
            weight_sum = np.sum(weights)
            if weight_sum > 0:
                for k in range(i, i + 2*W + 1):
                    for l in range(j, j + 2*W + 1):
                        if k < M + W and l < N + W:
                            image_channel_filtered[i, j] += (weights[k-i, l-j] * image_padded[k, l]) / weight_sum
    
    return image_channel_filtered

def non_local_means_filtering_color(image, W, sigma):
    """
    Apply Non-Local Means filtering to a color image.
    
    Parameters:
        image: 3D numpy array representing the color image (RGB)
        W: Integer, search window size
        sigma: Float, standard deviation of the noise
    
    Returns:
        image_filtered: 3D numpy array representing the filtered color image (RGB)
    """
    # Split the image into its color channels
    R, G, B = image[:, :, 0], image[:, :, 1], image[:, :, 2]
    
    # Apply the filtering to each channel
    R_filtered = non_local_means_filtering_channel(R, W, sigma)
    G_filtered = non_local_means_filtering_channel(G, W, sigma)
    B_filtered = non_local_means_filtering_channel(B, W, sigma)
    
    # Stack the channels back into a 3D array
    image_filtered = np.stack((R_filtered, G_filtered, B_filtered), axis=-1)
    
    return image_filtered

W = 10  # 예시 윈도우 크기
sigma = 10.0  # 예시 노이즈의 표준 편차

# 비국소 평균 필터

In [67]:
sigma_space = [4]
sigma_intensity = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

fn = "dog"
_noisy = '_noisy'
png = '.png'

filename = fn + png
clean_img = io.imread (filename)
clean_img = clean_img[:,:,0:3]
clean_img = clean_img/255.0

filename = fn + _noisy + png
noisy_img = io.imread(filename)
noisy_img = noisy_img[:,:,0:3]
noisy_img = noisy_img/255.0

for s_s in sigma_space:
	for s_i in sigma_intensity:
		bf = bilateral_filter(noisy_img, 3, s_s, s_i)
		print(f"kernel size=3, sigma_space={s_s}, sigma_intensity={s_i} - PSNR: {peak_signal_noise_ratio(clean_img, bf)}")


kernel size=3, sigma_space=4, sigma_intensity=1.0 - PSNR: 25.56380001512779


'\nfor s_s in sigma_space:\n\tfor s_i in sigma_intensity:\n\t\tbf = bilateral_filter(noisy_img, 3, s_s, s_i)\n\t\tprint(f"kernel size=3, sigma_space={s_s}, sigma_intensity={s_i} - PSNR: {peak_signal_noise_ratio(clean_img, bf)}")\n'

*** 빠른 양방향 필터 ***

In [85]:
def fff_bilateral_filter(noisy_img, k_size=5, sigma_space=4, sigma_intensity=0.2):
    h, w, ch = noisy_img.shape
    bilateral_noisy_img = np.zeros((h, w, ch))

    spatial_filter = gaussian_kernel(k_size, sigma_space)

    for c in range(ch):
        intensity_center = noisy_img[:, :, c]
        weighted_sum = np.zeros_like(intensity_center)
        normalization_factor = np.zeros_like(intensity_center)

        for m in range(-k_size//2, k_size//2 + 1):
            for n in range(-k_size//2, k_size//2 + 1):
                i_neighbors = np.clip(np.arange(h) + m, 0, h - 1)
                j_neighbors = np.clip(np.arange(w) + n, 0, w - 1)
                intensity_neighbors = noisy_img[i_neighbors, :, c][:, j_neighbors]
                weight_intensity = np.exp(-(intensity_center - intensity_neighbors)**2 / (2 * sigma_intensity**2))
                weight_spatial = spatial_filter[m + k_size//2, n + k_size//2]
                weighted_sum += intensity_neighbors * weight_intensity * weight_spatial
                normalization_factor += weight_intensity * weight_spatial

        bilateral_noisy_img[:, :, c] = weighted_sum / normalization_factor
    return bilateral_noisy_img

In [70]:
sigma_space = [4]
sigma_intensity = [0.01, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

fn = "dog"
_noisy = '_noisy'
png = '.png'

filename = fn + png
clean_img = io.imread (filename)
clean_img = clean_img[:,:,0:3]
clean_img = clean_img/255.0

filename = fn + _noisy + png
noisy_img = io.imread(filename)
noisy_img = noisy_img[:,:,0:3]
noisy_img = noisy_img/255.0

print("fast_bilateral_filter\n")
for s_s in sigma_space:
	for s_i in sigma_intensity:
		bf = fff_bilateral_filter(noisy_img, 5, s_s, s_i)
		print(f"kernel size=5, sigma_space={s_s}, sigma_intemsity={s_i} - PSNR: {peak_signal_noise_ratio(clean_img, bf)}")

fast_bilateral_filter

kernel size=5, sigma_space=4, sigma_intemsity=0.01 - PSNR: 14.235742101211581
kernel size=5, sigma_space=4, sigma_intemsity=0.1 - PSNR: 15.509620617675209
kernel size=5, sigma_space=4, sigma_intemsity=0.15 - PSNR: 16.8947055430603
kernel size=5, sigma_space=4, sigma_intemsity=0.2 - PSNR: 18.46346220024416
kernel size=5, sigma_space=4, sigma_intemsity=0.25 - PSNR: 20.11543463568749
kernel size=5, sigma_space=4, sigma_intemsity=0.3 - PSNR: 21.693411693786757
kernel size=5, sigma_space=4, sigma_intemsity=0.35 - PSNR: 23.002245028806087
kernel size=5, sigma_space=4, sigma_intemsity=0.4 - PSNR: 23.961350463343383
kernel size=5, sigma_space=4, sigma_intemsity=0.45 - PSNR: 24.61118795618692
kernel size=5, sigma_space=4, sigma_intemsity=0.5 - PSNR: 25.032026330667687
kernel size=5, sigma_space=4, sigma_intemsity=0.6 - PSNR: 25.458961405099252
kernel size=5, sigma_space=4, sigma_intemsity=0.7 - PSNR: 25.609732151321513
kernel size=5, sigma_space=4, sigma_intemsity=0.8 - P

In [81]:
fn = "dog"
_noisy = '_noisy'
png = '.png'

filename = fn + png
clean_img = io.imread (filename)
clean_img = clean_img[:,:,0:3]
clean_img = clean_img/255.0

filename = fn + _noisy + png
noisy_img = io.imread(filename)
noisy_img = noisy_img[:,:,0:3]
noisy_img = noisy_img/255.0

median_img = median_filter(noisy_img)
median_img_resized = resize(median_img, clean_img.shape, mode ='constant', anti_aliasing = True)
median_img_resized = np.clip(median_img_resized, 0., 1.0)

sigma_space = [3, 4, 5, 7]
sigma_intensity = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

print("fast_bilateral_filter\n")
for s_s in sigma_space:
	for s_i in sigma_intensity:
		bf = fff_bilateral_filter(noisy_img, 5, s_s, s_i)
		print(f"kernel size=5, sigma_space={s_s}, sigma_intemsity={s_i} - PSNR: {peak_signal_noise_ratio(clean_img, bf)}")

fast_bilateral_filter

kernel size=5, sigma_space=3, sigma_intemsity=0.1 - PSNR: 15.482560680105301
kernel size=5, sigma_space=3, sigma_intemsity=0.2 - PSNR: 18.392135237237245
kernel size=5, sigma_space=3, sigma_intemsity=0.3 - PSNR: 21.585433732029156
kernel size=5, sigma_space=3, sigma_intemsity=0.4 - PSNR: 23.871470681241483
kernel size=5, sigma_space=3, sigma_intemsity=0.5 - PSNR: 24.974357477097954
kernel size=5, sigma_space=3, sigma_intemsity=0.6 - PSNR: 25.425883603286312
kernel size=5, sigma_space=3, sigma_intemsity=0.7 - PSNR: 25.593061943578054
kernel size=5, sigma_space=3, sigma_intemsity=0.8 - PSNR: 25.642755359154297
kernel size=5, sigma_space=3, sigma_intemsity=0.9 - PSNR: 25.64498497109403
kernel size=5, sigma_space=3, sigma_intemsity=1.0 - PSNR: 25.628996811033005
kernel size=5, sigma_space=4, sigma_intemsity=0.1 - PSNR: 15.509620617675209
kernel size=5, sigma_space=4, sigma_intemsity=0.2 - PSNR: 18.46346220024416
kernel size=5, sigma_space=4, sigma_intemsity=0.3 - PSN

In [84]:
name = ['baby', 'bagles', 'beach', 'book', 'dog', 'girl_ani', 'lego', 'kitty', 'house', 'street']

for fn in name :
    _noisy = '_noisy'
    png = '.png'
    
    filename = fn + png
    clean_img = io.imread (filename)
    clean_img = clean_img[:,:,0:3]
    clean_img = clean_img/255.0
    
    filename = ""
    filename = fn + _noisy + png
    noisy_img = io.imread(filename)
    noisy_img = noisy_img[:,:,0:3]
    noisy_img = noisy_img/255.0
    
    title = filename + " denoising with median & fast_bilateral_filter\n"
    print(title)
    median_img = median_filter(noisy_img)
    median_img_resized = resize(median_img, clean_img.shape, mode ='constant', anti_aliasing = True)
    median_img_resized = np.clip(median_img_resized, 0., 1.0)
    sigma_intensity = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    for s_i in sigma_intensity:
        bf = fff_bilateral_filter(noisy_img, 5, 4, s_i)
        print(f"kernel size=5, sigma_space=4, sigma_intensity={s_i} - PSNR: {peak_signal_noise_ratio(clean_img, bf)}")
    print("\n")

baby_noisy.png denoising with median & fast_bilateral_filter

kernel size=5, sigma_space=4, sigma_intensity=0.5 - PSNR: 24.917477345880485
kernel size=5, sigma_space=4, sigma_intensity=0.6 - PSNR: 25.38608774501954
kernel size=5, sigma_space=4, sigma_intensity=0.7 - PSNR: 25.51589988473217
kernel size=5, sigma_space=4, sigma_intensity=0.8 - PSNR: 25.51963670560508
kernel size=5, sigma_space=4, sigma_intensity=0.9 - PSNR: 25.48017936752497
kernel size=5, sigma_space=4, sigma_intensity=1.0 - PSNR: 25.42929529175181


bagles_noisy.png denoising with median & fast_bilateral_filter

kernel size=5, sigma_space=4, sigma_intensity=0.5 - PSNR: 23.595119785131896
kernel size=5, sigma_space=4, sigma_intensity=0.6 - PSNR: 23.757910493370204
kernel size=5, sigma_space=4, sigma_intensity=0.7 - PSNR: 23.72953400148143
kernel size=5, sigma_space=4, sigma_intensity=0.8 - PSNR: 23.650406834694255
kernel size=5, sigma_space=4, sigma_intensity=0.9 - PSNR: 23.566149644006686
kernel size=5, sigma_space=4, s